# KddCup-AnomallyDetection


In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.

import numpy as np
import tensorflow as tf
import csv

    
        

Normalized Dataset saved as fullDataNormalized.csv file

Normalized Trainingset saved as trainingSetNormalized.csv file

Normalized Testset saved as testSetNormalized.csv file


In [2]:
with open('trainingSetNormalized.csv', 'rb') as f1:
    reader = csv.reader(f1)
    trainList = list(reader)
print("trainingSetNormalized.csv file opened")
    
with open('testSetNormalized.csv', 'rb') as f2:
    reader = csv.reader(f2)
    testList = list(reader)    
print("testSetNormalized.csv file opened")
print("OK!")

trainingSetNormalized.csv file opened
testSetNormalized.csv file opened
OK!


In [5]:
def file_len(adress):
    f = open(adress)
    nr_of_lines = sum(1 for line in f)
    f.close()
    return nr_of_lines

trainLen = file_len('trainingSetNormalized.csv')
print("Total length of train data : ", trainLen)

testLen = file_len('testSetNormalized.csv')
print("Total length of test data : ", testLen)

('Total length of train data : ', 787953)
('Total length of test data : ', 247256)


In [48]:
trainingSet = np.empty(shape=[trainLen, 41])
testSet = np.empty(shape=[testLen, 41]) 

for i in range(trainLen):
    for j in range(41):
        trainingSet[i][j] = trainList[i][j]
print("TrainingSet : ", trainingSet.shape)     
for i in range(trainLen):
    trainingSet[i][19] = 0

for i in range(testLen):
    for j in range(41):
        testSet[i][j] = testList[i][j] 
for i in range(testLen):
    testSet[i][19] = 0
print("TestSet : ", testSet.shape)        
#index 19 is nan because std is 0,,, so in matrix' 19th index element = 0

('TrainingSet : ', (787953, 41))
('TestSet : ', (247256, 41))


In [50]:
# Parameters
learning_rate = 0.01
training_epochs = 20
batch_size = 200
display_step = 1


# Network Parameters
n_input = 41
n_hidden_1 = 40 # 1st layer num features
n_hidden_2 = 35 # 2nd layer num features


# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([n_input])),
}

In [51]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),biases['encoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [52]:
# Launch the graph
# Using InteractiveSession (more convenient while using Notebooks)
sess = tf.InteractiveSession()
sess.run(init)
display_step = 300
num_steps=3001
total_batch = int(trainingSet.shape[0]/batch_size)
print("Total batch = ",total_batch)
# Training cycle
for step in range(num_steps):
# Pick an offset within the training data, which has been randomized.
# Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (trainingSet.shape[0])
# Generate a minibatch.
    batch_data = trainingSet[offset:(offset + batch_size), :]
    _, c = sess.run([optimizer, cost], feed_dict={X: batch_data})
    if step % display_step == 0:
        print("Epoch:", '%04d' % (step+1),"cost=", "{:.9f}".format(c))
        print("Step Count :" ,step);
        
print("Optimization Finished!")


('Total batch = ', 3939)
('Epoch:', '0001', 'cost=', '1.773986101')
('Step Count :', 0)
('Epoch:', '0301', 'cost=', '0.871800661')
('Step Count :', 300)
('Epoch:', '0601', 'cost=', '0.130277842')
('Step Count :', 600)
('Epoch:', '0901', 'cost=', '0.117659949')
('Step Count :', 900)
('Epoch:', '1201', 'cost=', '0.098911457')
('Step Count :', 1200)
('Epoch:', '1501', 'cost=', '0.116090700')
('Step Count :', 1500)
('Epoch:', '1801', 'cost=', '0.104804605')
('Step Count :', 1800)
('Epoch:', '2101', 'cost=', '0.109346196')
('Step Count :', 2100)
('Epoch:', '2401', 'cost=', '0.821393967')
('Step Count :', 2400)
('Epoch:', '2701', 'cost=', '0.105472445')
('Step Count :', 2700)
('Epoch:', '3001', 'cost=', '0.095400371')
('Step Count :', 3000)
Optimization Finished!


In [53]:
#encodeDecode = sess.run(y_pred, feed_dict={X: testData[:10000]})
encode_decode = sess.run(
    y_pred, feed_dict={X: testSet[:100]})

In [56]:
encode_decode.shape

(100, 41)

In [68]:
testSet[13]

array([ -7.67600000e-02,  -3.77861000e+00,  -2.48153000e+00,
        -5.50310000e-01,  -2.18000000e-03,  -3.33000000e-03,
        -4.98000000e-03,  -3.13500000e-02,  -1.71000000e-03,
        -5.40100000e-02,  -8.38000000e-03,  -1.30924000e+00,
        -6.75000000e-03,  -1.66600000e-02,  -9.16000000e-03,
        -1.19300000e-02,  -1.95300000e-02,  -1.82700000e-02,
        -5.83700000e-02,   0.00000000e+00,  -9.80000000e-04,
        -6.16400000e-02,   6.33930000e-01,   3.08658000e+00,
        -4.93780000e-01,  -4.94700000e-01,  -2.95180000e-01,
        -2.95580000e-01,   5.71720000e-01,  -2.70410000e-01,
        -4.42400000e-01,   9.08900000e-01,   8.56020000e-01,
         7.61810000e-01,  -3.71170000e-01,   4.18069000e+00,
        -3.91840000e-01,  -4.95270000e-01,  -4.93190000e-01,
        -3.06120000e-01,  -3.02600000e-01])

In [69]:
encode_decode[13]

array([  3.00228180e-06,   3.35547193e-05,   1.59903686e-06,
         5.88455350e-06,   4.86902463e-05,   2.98202591e-04,
         7.99740592e-05,   8.51102232e-06,   6.04503963e-04,
         1.03884468e-05,   3.89677698e-05,   7.87762880e-01,
         4.59654511e-05,   5.14870189e-05,   4.54760244e-04,
         4.27797379e-04,   2.76705487e-05,   1.80990290e-04,
         2.28837901e-03,   4.85179153e-05,   7.37562414e-06,
         2.05380256e-05,   8.88946772e-07,   2.17084438e-04,
         5.31000410e-07,   7.56725422e-06,   2.21778741e-04,
         2.21505252e-06,   6.37791634e-01,   2.51313686e-05,
         4.01417338e-07,   5.31951741e-07,   8.93490911e-01,
         7.71922886e-01,   1.66875748e-06,   9.99683619e-01,
         1.10073231e-01,   1.96198494e-06,   6.72947863e-06,
         8.62122700e-03,   4.34382207e-04], dtype=float32)

In [95]:
tempCost = 0
totalCost= 0
step = 0

for i in range(encode_decode.shape[0]):
    tempCost = np.mean(pow(testSet[i] - encode_decode[i], 2))
    if step % 10 == 0 :
        print("STEP :", '%04d' % (step+1),"cost = ", "{:.9f}".format(tempCost))
    step +=1
    totalCost+=tempCost
averageCost=float(totalCost/100)
print("Total cost : ", totalCost)
print("Average cost : ", averageCost)

('STEP :', '0001', 'cost = ', '1.176950030')
('STEP :', '0011', 'cost = ', '2.603309765')
('STEP :', '0021', 'cost = ', '0.993005706')
('STEP :', '0031', 'cost = ', '1.107182420')
('STEP :', '0041', 'cost = ', '2.761034306')
('STEP :', '0051', 'cost = ', '0.920144843')
('STEP :', '0061', 'cost = ', '0.964722185')
('STEP :', '0071', 'cost = ', '2.464248091')
('STEP :', '0081', 'cost = ', '0.854233926')
('STEP :', '0091', 'cost = ', '0.975328021')
('Total cost : ', 111.13502444422929)
('Average cost : ', 1.111350244442293)
